In [95]:
# Package used to insert a short wait time between data polls
import time

# Both the client and server must be running on the same machine 
# for 'local' operation.
from MultiPyVu import MultiVuClient as mvc
from MultiPyVu import MultiVuServer as mvs


# Start the server.
with mvs.MultiVuServer() as server:

    # Start the client
    with mvc.MultiVuClient() as client:
        
        time.sleep(5)
        
        # A basic loop that demonstrates communication between 
        # client/server
        client.set_field(200.0,
                         100, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
        # Wait for 20 seconds after temperature and field are stable
        print('Waiting ')
        client.wait_for(10,
                        timeout_sec=0)

        for t in range(5):
            # Polls MultiVu for the temperature, field, and their
            #  respective states
            T, sT = client.get_temperature()
            F, sF = client.get_field()

            # Relay the information from MultiVu
            message = f'The temperature is {T}, status is {sT}; '
            message += f'the field is {F}, status is {sF}. '
            print(message)

            # collect data at roughly 2s intervals
            time.sleep(2)

MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 56487)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 56487)


Waiting 
The temperature is 300.0057, status is Chasing; the field is 200.4971, status is Switch Warming. 
The temperature is 300.0042, status is Chasing; the field is 200.5259, status is Switch Warming. 
The temperature is 300.0066, status is Chasing; the field is 200.4971, status is Switch Warming. 
The temperature is 300.0075, status is Chasing; the field is 200.5402, status is Iterate. 
The temperature is 300.0057, status is Chasing; the field is 200.5546, status is Iterate. 


MultiVuServer - Client ('127.0.0.1', 56487) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 56487)
MultiVuServer - Closing connection to ('127.0.0.1', 56487)


In [1]:
import pyvisa as visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from labview_automation import LabVIEW

def nano_change_chanel(volt, chan=None):
    if not chan:
        return
    volt.write(":SENS:CHAN {}".format(chan))
    time.sleep(0.1)
    if volt.query(":SENS:CHAN?").strip() == str(chan):
        return
    else:
        print("channel switch error")
        raise

def init_hall_two_channel(dc_current=50e-6, step=3):
    for i in range(step):
        print("sample 1:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc1, volt1, dc_current=dc_current))
    print("-"*10)
    for i in range(step):
        print("sample 2:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc2, volt2, dc_current=dc_current))
        
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v


def measure_resistance_two_channel(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.05):
    #dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    
    dc.compliance_voltage = 100        # Sets the compliance voltage to 100 V
    
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_1.append(float(volt.query("fetch?")))
    v_up_1 = sorted(v_up_1)
    v_up_1 = v_up_1[3: -3]
    
    
    v_up_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_2.append(float(volt.query("fetch?")))
    v_up_2 = sorted(v_up_2)
    v_up_2 = v_up_2[3: -3]
    
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_1.append(float(volt.query("fetch?")))
        
        
    v_down_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_2.append(float(volt.query("fetch?")))
    
    dc.shutdown() 

    
    v_down_1 = sorted(v_down_1)
    v_down_1 = v_down_1[3: -3]
    v_down_2 = sorted(v_down_2)
    v_down_2 = v_down_2[3: -3]
    
    average_v_1 = (np.array(v_up_1).mean() - np.array(v_down_1).mean())/2
    std_v_1 = (np.array(v_up_1).std() + np.array(v_down_1).std())/2
    
    average_v_2 = (np.array(v_up_2).mean() - np.array(v_down_2).mean())/2
    std_v_2 = (np.array(v_up_2).std() + np.array(v_down_2).std())/2
    return average_v_1, std_v_1, average_v_2, std_v_2

import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise
        
def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
def generate_fields(fileds, steps):
    assert (len(fileds) == len(steps))
    res = []
    for i in range(len(fileds)):
        res.extend([f for f in range(fileds[i], fileds[i + 1] if i+1 < len(fileds) else 0, 
                                   -1 * steps[i])])
    res = res + [0] + [-1* f for f in res[::-1]]
    res += res[::-1][1:]
    return res  

def field_scan_two_sample_two_channel(dc1, volt1, dc2, volt2, dc_current_1=1e-4, dc_current_2=1e-4, fields=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        file_2 = "/data/" + sample2_id + "_" + str(T) + "K_" + str(int(dc_current_2*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        with open(file_2, 'w') as f:
            f.write("field(G), average_v_1, std_v_1,  average_v_2, std_v_2, dc_current\n")
        dc_current = None
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            for file in [file_1, file_2]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc1, volt=volt1, dc_current=dc_current)
                else:
                    dc_current = dc_current_2
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc2, volt=volt2, dc_current=dc_current)
                    
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, dc_current))

rm = visa.ResourceManager()
rm.list_resources()

('ASRL1::INSTR',
 'GPIB0::15::INSTR',
 'GPIB1::1::INSTR',
 'GPIB1::7::INSTR',
 'GPIB1::8::INSTR',
 'GPIB1::10::INSTR',
 'GPIB1::12::INSTR',
 'GPIB1::13::INSTR')

In [155]:
volt_ID=1
volt = rm.open_resource('GPIB1::{}::INSTR'.format(volt_ID))

def LVread():
    lv = LabVIEW()
    lv.start() # Launches the active LabVIEW with the listener VI
    with lv.client() as c:
        control_values = {
        
            }
    
        indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)
    #  print(indicators['Temp (K)'],indicators['Angle'], indicators['Field'] )
    #lv.kill() # Stop LabVIEW
    T=float(indicators['Temp (K)'])
    Ang=float(indicators['Angle'])
    Field=float(indicators['Field'])
    return T, Ang, Field


def measure_voltage_two_channel(volt):

    nano_change_chanel(volt, chan=1)
    vxx=float(volt.query("fetch?"))

    nano_change_chanel(volt, chan=2)
    vxy =float(volt.query("fetch?"))
    

    return vxx, vxy, float(time.time())

def measure_voltage_one_channel(volt):

    nano_change_chanel(volt, chan=1)
    vxx=float(volt.query("fetch?"))
    vxy=0
    

    return vxx, vxy, float(time.time())


def measure_voltage_one_channel_average(volt):

    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(0.1)
        if i < 10:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    
    vxx = np.array(v_up).mean()
    vxy=0    

    return vxx, vxy, float(time.time())

def WRITEvoltageinFile(volt, dc_current=0.1e-3, sample1_id="50nmWTe2",Temp='2K'):
        
#        Tmin, Tmax = Temp - 0.5, Temp + 0.5

        T = Temp
        Ang = -15    
        file = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file, 'w') as f:
            f.write("t, Ang, Field, vxx, vxy, dc_current, T\n")
        
        vxy, vxx, t0 = measure_voltage_two_channel(volt)
        
        lv = LabVIEW()
        lv.start() # Launches the active LabVIEW with the listener VI
        with lv.client() as c:
            control_values = {
        
                        }
            
            t1st = None
        
            while int(t0) < 5637029254:
        
                Ang=Ang+5
                print(Ang)
                time.sleep(3)
                        
            
                indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)

                T=float(indicators['Temp (K)'])
#                Ang=float(indicators['Angle'])
                Field=float(indicators['Field'])
                vxy, vxx, t = measure_voltage_two_channel(volt)
                print(vxx,vxy)           
               # (a, b)=divmod(int(t-t0),5)
     #       if b == 0:
      #          T, Ang, Field= LVread() 
                
#                 if round(T)==100:
#                     Ang=Ang+20
#                 if round(T)==10:
#                     Ang=Ang+30    
                
#                if not t1st & (T >= Tmin) & (T <= Tmax):
#                    t1st = t

#                if t1st & ((T < Tmin) | (T > Tmax)):
#                    t1st = None
#                    continue
#                else:
#                   if t - t1st >= 1800:
                with open(file, 'a') as f:
                     f.write("{},{},{},{},{},{},{}\n".format(t-t0, Ang, Field, vxx, vxy, dc_current, T))
                        

def WRITEvoltageinFile_onechannel(volt, dc_current=0.1e-3, sample1_id="50nmWTe2"):

        T = "10"
        Ang = -15
        file = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file, 'w') as f:
            f.write("t, Ang, Field, vxx, vxy, dc_current, T\n")
        
        vxx, vxy, t0 = measure_voltage_one_channel_average(volt)
        
        lv = LabVIEW()
        lv.start() # Launches the active LabVIEW with the listener VI
        with lv.client() as c:
            control_values = {
        
                        }
    
        
            while    int(t0) < 5637029254:
                
                Ang=Ang+5
                print(Ang)
                time.sleep(3)
                vxy, vxx, t = measure_voltage_one_channel_average(volt)
                print(vxx,vxy)

                        
            
                indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)

                T=float(indicators['Temp (K)'])
#               Ang=float(indicators['Angle'])
                Field=float(indicators['Field'])
                
               # (a, b)=divmod(int(t-t0),5)
     #       if b == 0:
      #          T, Ang, Field= LVread() 
                
                if round(T)==100:
                    Ang=Ang+20
                if round(T)==10:
                    Ang=Ang+30    
                    
                with open(file, 'a') as f:
                     f.write("{},{},{},{},{},{},{}\n".format(t-t0, Ang, Field, vxx, vxy, dc_current, T))


In [2]:
sensitivity = {1:26, 0.5:25, 0.2:24, 0.1:23, 0.05:22, 0.02:21, 0.01:20, 0.005:19, 0.002:18, 0.001:17, 0.0005:16, 0.0002:15,
               0.0001:14, 0.00005:13, 0.00002:12, 0.00001:11, 0.000005:10, 0.000002:9, 0.000001:8, 0.0000005:7, 0.0000002:6, 0.0000001:5, 0.00000005:4, 0.00000002:3,
               0.00000001:2, 0.000000005:1, 0.000000002:0}

timeconstant = {30000:19, 10000:18, 3000:17, 1000:16, 300:15, 100:14, 30:13, 10:12, 3:11, 1:10, 0.3:9, 0.1:8, 0.03:7,
                0.01:6, 0.003:5, 0.001:4, 0.0003:3, 0.0001:2, 0.00003:1, 0.00001:0}
import pyvisa as visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from labview_automation import LabVIEW

import pyvisa as visa
class Lockin:
    def __init__(self, resource, gpib=1):
        self.lock_in = resource.open_resource('GPIB1::{}::INSTR'.format(gpib))

    def change_harmonic(self, harm=1, sens=1):
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SCAL {}".format(sensitivity[sens]))
    
    def change_timeconstant(self, tc=1):
        self.lock_in.write("OFLT {}".format(timeconstant[tc]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 1").strip())
        y = float(self.lock_in.query("OUTP? 2").strip())
        r = float(self.lock_in.query("OUTP? 3").strip())
        theta = float(self.lock_in.query("OUTP? 4").strip())
        return x, y, r, theta

    def lock_in_measure(self, count=10, time_step=0.1,
                        wait_before_measure=1,
                        harm=1, sens=0.00001):
#        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)
            
        xs = sorted(xs)
        ys=  sorted(ys)
        rs=  sorted(rs)
        thetas=sorted(thetas)
        
        if len(xs)>30:
            xs = xs[round(1/6*len(xs)): -round(1/6*len(xs))]
            ys = ys[round(1/6*len(ys)): -round(1/6*len(ys))]
            rs = rs[round(1/6*len(rs)): -round(1/6*len(rs))]
            thetas = thetas[round(1/6*len(thetas)): -round(1/6*len(thetas))]

        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)
        
        
        return x_mean, y_mean, r_mean, theta_mean, \
               x_std, y_std,r_std, theta_std

rm = visa.ResourceManager()
rm.list_resources()

('ASRL1::INSTR',
 'GPIB0::15::INSTR',
 'GPIB1::1::INSTR',
 'GPIB1::7::INSTR',
 'GPIB1::8::INSTR',
 'GPIB1::10::INSTR',
 'GPIB1::12::INSTR',
 'GPIB1::13::INSTR')

In [6]:
lock3.change_harmonic(sens=0.1)

In [14]:
lock1.change_timeconstant(tc=1)

In [4]:
lock1 = Lockin(rm, 7)
lock2 = Lockin(rm, 8)
lock3 = Lockin(rm, 10)
lock4 = Lockin(rm, 11)

In [15]:
lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1)

(6.11883e-07, 1.42912e-05, 1.43052e-05, 87.54, 0.0, 0.0, 0.0, 0.0)

In [22]:
etrA, etrB, etrC, etrD = 0, 0, 1, 1

In [28]:
field_max, field_min = 200.0, -200.0
field_fine_max, field_fine_min = 50.0, -50.0
ramp_rate = 20.0
ramp_fine_rate = 5.0
sample_id = "BST_Mo_CoFeB_MgO_Ta"
T = 300
angle = 90
current = 6.0*1000

In [29]:
# Package used to insert a short wait time between data polls
import time

# Both the client and server must be running on the same machine 
# for 'local' operation.
from MultiPyVu import MultiVuClient as mvc
from MultiPyVu import MultiVuServer as mvs


file_name = "QS/" + sample_id + "_" + str(T) + "K_" + str(angle) + "deg_"+ str(current) + "uA_" + str(int(time.time())) + ".csv"
with open(file_name, 'a') as f:
        f.write("{},{},{},{},{}, {}, {}, {}, {}\n".format('field', 'Rxx1', 'field', 'Rxy1', 'field', 'Rxx2', 'field', 'Rxy2', 'Temp'))
        

# Start the server.
with mvs.MultiVuServer() as server:

    # Start the client
    with mvc.MultiVuClient() as client:
        
        time.sleep(5)
        
        # A basic loop that demonstrates communication between 
        # client/server
        client.set_field(field_max,
                         ramp_rate, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
        F, sF = client.get_field()
        Flg = 1
        while  Flg:
            F = client.get_field()
            if F[0] > field_max:
                Flg = 0
        
        # Wait for 5 seconds
        print('Waiting ')
        client.wait_for(5,
                        timeout_sec=0)
        client.set_field(field_min,
                         ramp_rate, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
        
        Flg = 1
        fine_Flg = 0
        Flg_set = 0
        # ramp down
        print('ramp down\n')
        while Flg:
            F1 = client.get_field()
            A = lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F2 = client.get_field()
            B = lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F3 = client.get_field()
            C = lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F4 = client.get_field()
            D = lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F5 = client.get_field()
            #F2 = client.get_field()

            
            Fxx1 = (F1[0] + F2[0])/2
            Fxy1 = (F2[0] + F3[0])/2
            Fxx2 = (F3[0] + F4[0])/2
            Fxy2 = (F4[0] + F5[0])/2
            
            Temp = client.get_temperature()
            with open(file_name, 'a') as f:
                f.write("{},{},{},{},{}, {}, {}, {}, {}\n".format(Fxx1, A[etrA], Fxy1, B[etrB], Fxx2, C[etrC], Fxy2, D[etrD], Temp[0]))
                #f.write("{},{},{},{},{}\n".format(F, A[0], B[0], C[2], D[2]))
        
            
            
            if not Flg_set:
                if F5[0] < (field_fine_max + 100.0) and F5[0] > field_fine_min: # +100.0 allows the field to decelerate
                    fine_Flg = 1
                    Flg_set = 1
                    print('fine flg raised\n')
                # change the ramping rate to fine ramping rate
                if fine_Flg:
                    print('speed redueced\n')
                    client.set_field(field_min,
                             ramp_fine_rate, 
                             client.field.approach_mode.linear,
                             client.field.driven_mode.persistent)
        
            if Flg_set:
                if F5[0] < field_fine_min:
                    fine_Flg = 0
                    Flg_set = 0
                    print('fine flg lowered\n')
                if not fine_Flg:
                    print('speed increased\n')
                    client.set_field(field_min,
                             ramp_rate, 
                             client.field.approach_mode.linear,
                             client.field.driven_mode.persistent)
        
            if F5[0] < field_min:
                    Flg = 0

        
        # Wait for 5 seconds
        print('Waiting ')
        client.wait_for(5,
                        timeout_sec=0)
        client.set_field(field_max,
                         ramp_rate, 
                         client.field.approach_mode.linear,
                         client.field.driven_mode.persistent)
        
        Flg = 1
        fine_Flg = 0
        Flg_set = 0
        # ramp up
        print('ramp up\n')
        while Flg:
            F1 = client.get_field()
            A = lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F2 = client.get_field()
            B = lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F3 = client.get_field()
            C = lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F4 = client.get_field()
            D = lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F5 = client.get_field()
            #F2 = client.get_field()
            
            Fxx1 = (F1[0] + F2[0])/2
            Fxy1 = (F2[0] + F3[0])/2
            Fxx2 = (F3[0] + F4[0])/2
            Fxy2 = (F4[0] + F5[0])/2
            
            Temp = client.get_temperature()
            with open(file_name, 'a') as f:
                f.write("{},{},{},{},{}, {}, {}, {}, {}\n".format(Fxx1, A[etrA], Fxy1, B[etrB], Fxx2, C[etrC], Fxy2, D[etrD], Temp[0]))
            
            
            '''F1 = client.get_field()
            A = lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            B = lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            C = lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            D = lock4.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0)
            F2 = client.get_field()
            F = (F1[0] + F2[0])/2
            
            with open(file_name, 'a') as f:
                f.write("{},{},{},{},{}\n".format(F, A[0], B[0], C[2], D[2]))'''
        
            
            
            if not Flg_set:
                if F5[0] < field_fine_max and (F5[0] > field_fine_min - 100.0): # -100.0 allows the field to decelerate
                    fine_Flg = 1
                    Flg_set = 1
                    print('fine flg raised\n')
                # change the ramping rate to fine ramping rate
                if fine_Flg:
                    print('speed redueced\n')
                    client.set_field(field_max,
                             ramp_fine_rate, 
                             client.field.approach_mode.linear,
                             client.field.driven_mode.persistent)
        
            if Flg_set:
                if F5[0] > field_fine_max:
                    fine_Flg = 0
                    Flg_set = 0
                    print('fine flg lowered\n')
                if not fine_Flg:
                    print('speed increased\n')
                    client.set_field(field_max,
                             ramp_rate, 
                             client.field.approach_mode.linear,
                             client.field.driven_mode.persistent)
        
            if F5[0] > field_max:
                    Flg = 0
        
        
        
        
        '''Flg = 1
        while Flg:
            F1 = client.get_field()
            B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1)
            F2 = client.get_field()
            F = (F1[0] + F2[0])/2
            
            print(F, B[0])
            
            
            if F2[0] < 200.0:
                microwave.write(":POW:STAT OFF;")
                Flg = 0'''

                
        for t in range(5):
            # Polls MultiVu for the temperature, field, and their
            #  respective states
            T, sT = client.get_temperature()
            F, sF = client.get_field()

            # Relay the information from MultiVu
            message = f'The temperature is {T}, status is {sT}; '
            message += f'the field is {F}, status is {sF}. '
            print(message)

            # collect data at roughly 2s intervals
            time.sleep(2)

MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 50557)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 50557)
--- Logging error ---
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\ProgramData\Anaconda3\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\User Data\\Cheng Yang\\Sam files\\_switch\\QdMultiVu.log' -> 'D:\\User Data\\Cheng Yang\\Sam files\\_switch\\QdMultiVu.log.1'
Call stack:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", 

Waiting 
ramp down

fine flg raised

speed redueced

fine flg lowered

speed increased

Waiting 
ramp up

fine flg raised

speed redueced

fine flg lowered

speed increased

The temperature is 300.027, status is Stable; the field is 200.0655, status is Switch Cooling. 
The temperature is 300.0267, status is Stable; the field is 200.0655, status is Switch Cooling. 
The temperature is 300.0291, status is Stable; the field is 200.0511, status is Switch Cooling. 
The temperature is 300.0302, status is Stable; the field is 200.0511, status is Switch Cooling. 
The temperature is 300.0279, status is Stable; the field is 200.0511, status is Switch Cooling. 


MultiVuServer - Client ('127.0.0.1', 50557) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 50557)
MultiVuServer - Closing connection to ('127.0.0.1', 50557)


In [13]:
import time

file_name = "QS/" + 'hmmm' + "_" + str(300) + "K_" + str(int(time.time())) + ".csv"
with open(file_name, 'a') as f:
        f.write("{},{},{},{},{}\n".format('field', 'Rxx1', 'Rxy1', 'Rxx2', 'Rxy2'))

with open(file_name, 'a') as f:
    f.write("{},{},{},{},{}\n".format(1, 2, 3, 4, 5))